In [20]:
import os
import sys
import urllib.request
import json
import pandas as pd

def get_naver_news(query, display=10, start=1, sort='sim'):
    client_id = "cocNOmP0Kiwk8PKGSvJb"
    client_secret = "U76utjq_lW"
    encText = urllib.parse.quote(query)
    
    url = f"https://openapi.naver.com/v1/search/news?query={encText}&display={display}&start={start}&sort={sort}"
    
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    
    try:
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read()
            return json.loads(response_body.decode('utf-8'))
        else:
            print(f"Error Code: {rescode}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
all_items = []
query = input("검색어를 입력하세요: ")  # 사용자로부터 검색어 입력 받기
max_results_per_page = 100 # display 최대값
max_start = 1000 # start 최대값

# 정확도순 데이터 수집
for start_idx in range(1, max_start + 1, max_results_per_page):
    data = get_naver_news(query, display=max_results_per_page, start=start_idx, sort='sim')
    if data and data['items']:
        all_items.extend(data['items'])
    else:
        break # 더 이상 데이터가 없으면 중단

# # 날짜순 데이터 수집 (중복 제거를 위해 따로 수집 후 합칠 수 있음)
# for start_idx in range(1, max_start + 1, max_results_per_page):
#     data = get_naver_news(query, display=max_results_per_page, start=start_idx, sort='date')
#     if data and data['items']:
#         all_items.extend(data['items'])
#     else:
#         break

# 중복된 항목 제거 (originallink 기준으로 제거)
unique_items = {item['originallink']: item for item in all_items}.values()

df = pd.DataFrame(list(unique_items))

# 불필요한 HTML 태그 제거 및 날짜 형식 통일
df['title'] = df['title'].str.replace('<b>', '').str.replace('</b>', '').str.replace('&quot;', '')
df['description'] = df['description'].str.replace('<b>', '').str.replace('</b>', '').str.replace('&quot;', '')
df['pubDate'] = pd.to_datetime(df['pubDate'], format='%a, %d %b %Y %H:%M:%S +0900')


print(df.head())
print(f"\n총 수집된 뉴스 기사 수: {len(df)}")

                                       title  \
0    SKT 신규 영업재개 첫날 되찾은 가입자 규모 고작 257명…무슨 일?   
1              HBM으로 삼성 제친 SK, 재계 1위 자리도 노린다   
2  [단독] 법상 자회사인데 ‘회계는 남남’…삼성생명, 수조원 부채 미반...   
3     삼성SDS, AI 잡고 계열사 의존 줄인다…외부 매출 10%P↑ 목표   
4    삼성·삼양·파마리서치, 인적분할로 지배구조 재편…평가 엇갈린 이유...   

                                        originallink  \
0  https://www.newsis.com/view/NISX20250625_00032...   
1  https://www.nocutnews.co.kr/news/6360245?utm_s...   
2  https://www.hani.co.kr/arti/economy/marketing/...   
3  https://www.ngetnews.com/news/articleView.html...   
4  http://www.edaily.co.kr/news/newspath.asp?news...   

                                                link  \
0  https://n.news.naver.com/mnews/article/003/001...   
1  https://n.news.naver.com/mnews/article/079/000...   
2  https://n.news.naver.com/mnews/article/028/000...   
3  https://www.ngetnews.com/news/articleView.html...   
4  https://n.news.naver.com/mnews/article/018/000...   

                                     

In [21]:
df.sort_values(by='pubDate', ascending=False)

,title,originallink,link,description,pubDate
564,"코스피 3,080선 강보합…코스닥 0.13% 상승세",https://www.topstarnews.net/news/articleView.h...,https://www.topstarnews.net/news/articleView.h...,"삼성전자(0.66%)가 소폭 오르고 있는 가운데, 전일 강세를 보인 SK하이닉스는 ...",2025-06-27 09:54:00
4,"삼성·삼양·파마리서치, 인적분할로 지배구조 재편…평가 엇갈린 이유...",http://www.edaily.co.kr/news/newspath.asp?news...,https://n.news.naver.com/mnews/article/018/000...,분리한다고 공시했다. 삼성바이오로직스의 분할 전후 지배구조 (자료=삼성바이오로직스)...,2025-06-27 09:21:00
55,"[코스피·코스닥,셀트리온 롯데지주 LS전선 코오롱생명과학 한화에어로...",http://www.issuenbiz.com/news/articleView.html...,http://www.issuenbiz.com/news/articleView.html...,"주요공시] ▲ 셀트리온(068270), 1000억원 규모 자사주 취득 결정 ▲ 솔루...",2025-06-27 07:56:00
21,"시스템반도체 관련주, '맑음' 미래반도체·한미반도체·SK하이닉스·테크...",http://www.finomy.com/news/articleView.html?id...,http://www.finomy.com/news/articleView.html?id...,"반면 이미지스, 삼성전자, 하나마이크론, 고영은 하락했다. 이미지스와 삼성전자는 전...",2025-06-26 18:06:00
3,"삼성SDS, AI 잡고 계열사 의존 줄인다…외부 매출 10%P↑ 목표",https://www.ngetnews.com/news/articleView.html...,https://www.ngetnews.com/news/articleView.html...,"금융감독원 전자공시시스템에 따르면, 삼성SDS는 지난해 총매출 5조4474억원 중 ...",2025-06-26 17:08:00
...,...,...,...,...,...
943,삼성전자가 밸류업 공시한다면...'밸류업 지수'가 밸류업 하는 길,http://www.fnnews.com/news/202410311131486059,https://n.news.naver.com/mnews/article/014/000...,증권업계 관계자는 최근 수익성과 성장성에서 의문이 제기되고 있는 삼성전자가 밸류업 ...,2024-10-31 14:06:00
948,"[이슈분석] 삼성전자, 밸류업 공시 '0건'…재계, 망설이는 까닭은",http://www.opinionnews.co.kr/news/articleView....,http://www.opinionnews.co.kr/news/articleView....,"4대그룹 중 유일, 밸류업 공시 전무한 삼성전자 30일 금융감독원 전자공시시스템에 ...",2024-10-30 17:42:00
949,활발해진 밸류업 공시···'대장주' 삼성전자는 언제 나올까,https://www.sisajournal-e.com/news/articleView...,https://www.sisajournal-e.com/news/articleView...,이 같은 상황에서 국내 증시의 대표주인 삼성전자의 밸류업 공시가 언제 나올지 투자자...,2024-10-29 16:28:00
950,"4대그룹 '밸류업' 공시, SK 앞서가다",https://www.fetv.co.kr/news/article.html?no=17...,https://www.fetv.co.kr/news/article.html?no=17...,"다만 4대 그룹 중 최대 규모의 기업인 삼성전자는 밸류업 공시는 물론이며, 밸류업 ...",2024-10-29 10:16:00
